# Proceso estándar de regresión

1. Separar datos de entrenamiento / prueba
2. Normalizar datos / Crear un normalizador
3. Crear modelo (NN)
4. Entrenar
5. Evaluar resultados

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Red neuronal secuencial
from tensorflow.keras.models import Sequential
# Capa completamente conectada
from tensorflow.keras.layers import Dense, Normalization
# Optimizador
from tensorflow.keras.optimizers import Adam

In [3]:
# asegurar que los resultados sean "reproducibles"
import tensorflow as tf

np.random.seed(1234)
tf.random.set_seed(1234)

### Datos de entrenamiento / prueba

In [10]:

x_train = pd.read_csv('data_train.csv')
#seleciona la ultima columna MPG
y_train = x_train.loc[:,'MPG']
#seleciona la toda informacion sin la ultima columna MPG
x_train=x_train.iloc[:,0:9]

x_test = pd.read_csv('data_test.csv')
y_test = x_test.loc[:,'MPG']
x_train=x_train.iloc[:,0:9]

   Cylinders  Displacement  Horsepower  Weight  Acceleration  Model Year  \
0          4            90          75    2125          14.5          74   
1          4           140          88    2890          17.3          79   
2          8           350         160    4456          13.5          72   
3          4           105          63    2125          14.7          82   
4          4            97          67    2145          18.0          80   

   Europe  Japan  USA  
0       0      0    1  
1       0      0    1  
2       0      0    1  
3       0      0    1  
4       0      1    0  
0      28.0
1      22.3
2      12.0
3      38.0
4      33.8
       ... 
309    19.8
310    16.0
311    26.0
312    32.0
313    28.0
Name: MPG, Length: 314, dtype: float64


In [2]:

y_train = pd.read_csv('y_train.csv')
X_test = pd.read_csv('X_test.csv')
y_test = pd.read_csv('y_test.csv')

FileNotFoundError: ignored

### Normalización

In [ ]:
def crear_normalizador(datos):
    normalizer = Normalization(axis=-1)
    normalizer.adapt(np.array(datos))
    return normalizer

In [ ]:
normalizador = crear_normalizador(X_train)

### Creación Red Neuronal

In [ ]:
def crear_nn_regresion(norm):
    model = Sequential([
        norm,
        Dense(32, activation='relu'),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(1)
    ])
    model.compile(loss='mean_absolute_error', optimizer=Adam(0.001))
    return model

In [ ]:
nn = crear_nn_regresion(normalizador)

### Entrenamiento del modelo

In [ ]:
history = nn.fit(
    X_train,
    y_train,
    validation_split=0.2,
    verbose=0,
    epochs=100
)

### Evaluar el rendimiento del modelo

In [ ]:
nn.evaluate(X_test, y_test, verbose=0)

In [ ]:
# Creación de predicciones
y_pred = nn.predict(X_test).flatten()

In [ ]:
a = plt.axes(aspect='equal')
plt.scatter(y_test, y_pred)
plt.xlabel('Real [MPG]')
plt.ylabel('Prediccion [MPG]')
lims = [0, 50]
plt.xlim(lims)
plt.ylim(lims)
plt.plot(lims, lims)